In [1]:
import numpy as np
import os
import rasterio
from osgeo import gdal

In [2]:
import scipy.ndimage

In [3]:
%load_ext cython

## Notes

This notebook contains code used in developing and applying a topographic occlusion correct to EVI and other BRDF imagery. 

We observed anomalous EVI values in steep mountainous valleys, running generally east-west, in higher latitudes in winter months. We believe these are due to some areas being in permanent or near-permanent shadow under these conditions. 

We applied a two-stage identification process, operating on the monthly means dataset (one image for all Januaries, all Februaries, etc):
* Identify areas likely to be affected by running a hillshade analysis with appropriate sun positions (due south in northern hemisphere and vice versa), and select pixels with illumination below an empiricially-determined threshold
* Identify pixels  where mean EVI values in winter were higher than those in "shoulder months" e.g. April

The pixels selected by both of these processes were identified. We then applied the following adjustment to those pixels:
* Where the mean for e.g. January was higher than the corresponding shoulder-season month (e.g. April) then set the January value to be equal to the April value
* Adjust the synoptic (all-images) standard deviation value for that pixel to exclude the e.g. January value and double-count the e.g. April value.

Gapfilling was then run as normal using the adjusted mean and standard deviation images, such that pixels from the 8-daily imagery affected by the occlusion issue would then be more likely to be removed by the despeckle algorithm (as the net effect of the topographic correction will have been to lower the mean and reduce the standard deviation of those pixels, thus higher 8-daily pixels would be more likely to be more than 1.96 s.d. from the mean).

In [4]:
# 13 band images, one for each month plus one synoptic
inMeansFile = r'G:\NewStats\EVI_Monthly_Means.tif'
inSDFile = r'G:\NewStats\RepeatMonthlySDs\EVI_Monthly_SDs.tif'
inCountFile = r'G:\NewStats\EVI_Monthly_Counts.tif'


### Notes on identifying occluded areas

* Two hillshade images were calculated: one for use in the northern hemisphere, with illumination from 180 deg due south; 
the other for use in the southern hemisphere with illumination from 360 deg due north. These are byte type, values from 0-255.

* These were calculated on 500m (15 arcsec) data and then aggregated to 1km taking the minimum, i.e. least illuminated, of the 4 input cells.

* For the areas N/S of +- 30 deg latitude, we mark as possibly occluded those parts of the respective hillshades with illumination
below a certain threshold... empirically 120 seems about right. 

* The flagged occluded pixels were then expanded using a binary dilation procedure, to expand the area where the check could run. (Noting that not all pixels marked as occluded will be modified: only if they also have winter > summer means.)

* Combine these into a single raster and output it. We will use this to mask the areas where the mean seasonality check runs, so as not 
to false-positive in large areas where winter rain actually does green things up.



In [1]:
# hillshade reprocessed to binary image identifying pixels above/below an empirically determined threshold
inOcclusionFile = r'C:\Users\zool1301\Documents\Other_Data\Ferranti_Elev_15Sec\Hillshade_Resample\HS_Comb_LT120.tif'


In [25]:
# Combine the northern and southern threshold images

occlusionThreshold = 120 # 95 gets turkmenistan stuff but not smaller mountatins like alaska and alps

nLimit = 7200 # 30 deg north
sLimit = 14400 # 30 deg south

inHillShadeNorthFile = r'C:\Users\zool1301\Documents\Other_Data\Ferranti_Elev_15Sec\Hillshade_Resample\HS180Deg_1k_5km_Min.tif'
inHillShadeSouthFile = r'C:\Users\zool1301\Documents\Other_Data\Ferranti_Elev_15Sec\Hillshade_Resample\HS360Deg_1k_5km_Min.tif'
inHSNorthDS = gdal.Open(inHillShadeNorthFile)
inHSNorthBand = inHSNorthDS.GetRasterBand(1)
inHSNorth = inHSNorthBand.ReadAsArray()
inHSSouthDS = gdal.Open(inHillShadeSouthFile)
inHSSouthBand = inHSSouthDS.GetRasterBand(1)
inHSSouth = inHSSouthBand.ReadAsArray()

inHSNorth = inHSNorth < occlusionThreshold
inHSSouth = inHSSouth < occlusionThreshold

occludedAreas = np.zeros(shape = inHSNorth.shape, dtype = np.byte)
occludedAreas[0:nLimit] = inHSNorth[0:nLimit]
occludedAreas[sLimit:] = inHSSouth[sLimit:]

outDrv = gdal.GetDriverByName('GTiff')
outputOcclusion = outDrv.Create('C:\Users\zool1301\Documents\Other_Data\Ferranti_Elev_15Sec\Hillshade_Resample\HS_Comb_LT{0!s}.tif'
                                .format(occlusionThreshold),
                        43200,21600,1,gdal.GDT_Byte,
                        ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES","INTERLEAVE=BAND","COMPRESS=LZW","PREDICTOR=2"])
outputOcclusion.SetGeoTransform(inHSNorthDS.GetGeoTransform())
outputOcclusion.SetProjection(inHSNorthDS.GetProjection())

outBand = outputOcclusion.GetRasterBand(1)
outBand.WriteArray(occludedAreas)
outBand.FlushCache()
outBand = None
outputOcclusion = None


In [35]:
with rasterio.open(inMeansFile) as src:
    above30N = int((90 - 30) / src.res[0])
    below30S = int((90 + 30) / src.res[0])
    w60 = int(60 / src.res[1])
    w90 = int(90 / src.res[1])
    xSize = src.width
src = gdal.Open(inMeansFile)
globalGT = src.GetGeoTransform()
globalProj = src.GetProjection()
NDV = src.GetRasterBand(1).GetNoDataValue()
src = None

In [9]:
struct_8 = scipy.ndimage.generate_binary_structure(2,2)

In [5]:
inOcclusionds = gdal.Open(inOcclusionFile)
#occludedAreas = inOcclusionds.ReadAsArray()
   

In [ ]:
inOcclusionds.ReadAsArray()

In [31]:
occludedAreas_Nbr1 = scipy.ndimage.binary_dilation(occludedAreas, structure=struct_8).astype(occludedAreas.dtype)

In [36]:
outDrv = gdal.GetDriverByName('GTiff')
outputOcclusion = outDrv.Create('C:\Users\zool1301\Documents\Other_Data\Ferranti_Elev_15Sec\Hillshade_Resample\HS_Comb_LT120_Nbr1.tif',
                        43200,21600,1,gdal.GDT_Byte,
                        ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES","INTERLEAVE=BAND","COMPRESS=LZW","PREDICTOR=2"])
outputOcclusion.SetGeoTransform(globalGT)
outputOcclusion.SetProjection(globalProj)

outBand = outputOcclusion.GetRasterBand(1)
outBand.WriteArray(occludedAreas_Nbr1)
outBand.FlushCache()
outBand = None
outputOcclusion = None


In [37]:
occludedAreas_Nbr2 = scipy.ndimage.binary_dilation(occludedAreas_Nbr1, structure=struct_8).astype(occludedAreas.dtype)

In [39]:
occludedAreas_Nbr2.max()

1

In [40]:
outDrv = gdal.GetDriverByName('GTiff')
outputOcclusion = outDrv.Create('C:\Users\zool1301\Documents\Other_Data\Ferranti_Elev_15Sec\Hillshade_Resample\HS_Comb_LT120_Nbr2.tif',
                        43200,21600,1,gdal.GDT_Byte,
                        ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES","INTERLEAVE=BAND","COMPRESS=LZW","PREDICTOR=2"])
outputOcclusion.SetGeoTransform(globalGT)
outputOcclusion.SetProjection(globalProj)

outBand = outputOcclusion.GetRasterBand(1)
outBand.WriteArray(occludedAreas_Nbr2)
outBand.FlushCache()
outBand = None
outputOcclusion = None

In [41]:
inOcclusionFile_Nbr2 = r'C:\Users\zool1301\Documents\Other_Data\Ferranti_Elev_15Sec\Hillshade_Resample\HS_Comb_LT120_Nbr2.tif'

In [46]:
above30N, below30S

(7200, 14400)

In [47]:
xCorners = np.linspace(0,xSize,6).astype(np.int32)

## Applying the corrections

Based on the occlusion dataset produced above, the identification and modification process proceeded as follows:

In [ ]:
suitableWidth = 4320
edges = np.arange(0, xSize, suitableWidth)
slices = zip(edges[:-1], edges[1:])
slices[-1] = (slices[-1][0], xSize)

#globalGT = inMeansds.GetGeoTransform()
#globalProj = inMeansds.GetProjection()
outDrv = gdal.GetDriverByName('GTiff')
#ndv = bnd.GetNoDataValue()

outMeanRaster = outDrv.Create(r'C:\Users\zool1301\AppData\Local\Temp\EVI_Monthly_Means_WinterCut_OcclusionMasked_Nbr2.tif',
                        43200,21600,13,gdal.GDT_Float32,
                        ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES","INTERLEAVE=BAND","COMPRESS=LZW","PREDICTOR=2"])
outSDRaster = outDrv.Create(r'C:\Users\zool1301\AppData\Local\Temp\EVI_Monthly_SDs_WinterCut_OcclusionMasked_Nbr2.tif',
                        43200,21600,13,gdal.GDT_Float32,
                        ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES","INTERLEAVE=BAND","COMPRESS=LZW","PREDICTOR=2"])
outCountRaster = outDrv.Create(r'C:\Users\zool1301\AppData\Local\Temp\EVI_Monthly_Count_WinterCut_OcclusionMasked_Nbr2.tif',
                        43200,21600,13,gdal.GDT_Int16,
                        ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES","INTERLEAVE=BAND","COMPRESS=LZW","PREDICTOR=2"])

flagsRaster = outDrv.Create(r'C:\Users\zool1301\AppData\Local\Temp\EVI_Monthly_Means_WinterCutFlags_OcclusionMasked_Nbr2.tif',
                     43200,21600,12,gdal.GDT_Byte,
                     ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES","INTERLEAVE=BAND","COMPRESS=LZW","PREDICTOR=2"])

outMeanRaster.SetGeoTransform(globalGT)
outMeanRaster.SetProjection(globalProj)
outSDRaster.SetGeoTransform(globalGT)
outSDRaster.SetProjection(globalProj)
outCountRaster.SetGeoTransform(globalGT)
outCountRaster.SetProjection(globalProj)

flagsRaster.SetGeoTransform(globalGT)
flagsRaster.SetProjection(globalProj)

for west, east in slices:
    print "{0!s} - {1!s} loading...".format(west,east),
    inMeansDS = gdal.Open(inMeansFile)
    meanDataStack = inMeansDS.ReadAsArray(west, 0, east-west)#[:-1]
    inMeansDS = None
    inSdDS =  gdal.Open(inSDFile)
    sdDataStack = inSdDS.ReadAsArray(west, 0, east-west)#[:-1]
    inSdDS = None
    inCountDS = gdal.Open(inCountFile)
    countDataStack = inCountDS.ReadAsArray(west, 0, east-west)#[:-1]
    inCountDS = None
    #inOcclusionds = gdal.Open(inOcclusionFile)
    inOcclusionds = gdal.Open(inOcclusionFile_Nbr2)
    occludedAreas = inOcclusionds.ReadAsArray(west, 0, east-west)
    inOcclusionds = None
    
    print "Running...",
    resFlags = ApplyWinterMinCut(meanDataStack, sdDataStack, countDataStack, occludedAreas, NDV)
    print "Saving...",
    for b in range(13):
        print "{0!s}... ".format(b),
        outMeanRaster.GetRasterBand(b+1).WriteArray(meanDataStack[b], west, 0)
        outSDRaster.GetRasterBand(b+1).WriteArray(sdDataStack[b], west, 0)
        outCountRaster.GetRasterBand(b+1).WriteArray(countDataStack[b], west, 0)
        
        if b < 12:
            flagsRaster.GetRasterBand(b+1).WriteArray(resFlags[b], west, 0)
        
        outMeanRaster.GetRasterBand(b+1).SetNoDataValue(NDV)
        outSDRaster.GetRasterBand(b+1).SetNoDataValue(NDV)
        outCountRaster.GetRasterBand(b+1).SetNoDataValue(NDV)
        
        if b < 12:
            flagsRaster.GetRasterBand(b+1).SetNoDataValue(NDV)
        
        #outputRaster.GetRasterBand(b+1).WriteArray(res[0][b], west, 0)
        #flagsRaster.GetRasterBand(b+1).WriteArray(res[1][b], west, 0)
        #outputRaster.GetRasterBand(b+1).SetNoDataValue(NDV)
        #flagsRaster.GetRasterBand(b+1).SetNoDataValue(NDV)
    print 
flagsRaster.FlushCache()
outMeanRaster.FlushCache()
outSDRaster.FlushCache()
outCountRaster.FlushCache()

flagsRaster = None
outMeanRaster = None
outSDRaster = None
outCountRaster = None


In [ ]:
%%cython
import numpy as np
from libc.math cimport sqrt

cpdef RegenerateGrandStats(float[:] groupMeans, float[:] groupSDs, short[:] groupCounts, float NDV):
    '''
    Recalculates "grand standard deviation" (and mean) from the group SDs, means, and counts. 
    
    For use to recalculate "SD_From_Daily" when we have modified some of the "Monthly_SDs" 
    e.g. decided to exclude the January SD value at a particular location if it was shaded. 
    
    In such a case we need to recalculate the SD that was produced from all daily values, 
    to also reflect the exclusion of the January days, but don't want to go back through all the 
    daily data to do so.    
    
    See
    http://www.burtonsys.com/climate/composite_sd.php#python
    '''
    
    assert groupMeans.shape[0] == groupSDs.shape[0]
    assert groupMeans.shape[0] == groupCounts.shape[0]
    
    cdef:
        Py_ssize_t length, i
        int N = 0
        double GM = 0.0
        
        double ESS = 0.0
        double TGSS = 0.0
        
        double G_STD = 0.0
        
    length = groupMeans.shape[0]
    
    for i in range(length):
        if groupCounts[i] != 0 and groupMeans[i] != NDV:
            N += groupCounts[i]
            GM += groupMeans[i] * groupCounts[i]
            ESS += ((groupSDs[i])**2) * (groupCounts[i]-1)
    
    #print "group mean sum {0!s}".format(GM)
    if N > 1:
        GM = GM / N
    #print "GM {0!s}".format(GM)
    
    for i in range(length):
        if groupCounts[i] != 0 and groupMeans[i] != NDV:
            TGSS += ((groupMeans[i] - GM)**2) * groupCounts[i]
    
    if N > 1:
        G_STD = sqrt((ESS+TGSS) / (N-1))
    
    return (GM, G_STD, N)


cpdef ApplyWinterMinCut(float[:,:,::1] meansData, float[:,:,::1] sdData, short[:,:,::1] countData,
                        char[:,::1] occludedAreas, float NDV):

    '''
    Applies topographic occlusion "correction" as described in method notes.
    
    Filters monthly mean / standard deviation / count values to remove pixels that are higher 
    in winter than in summer, if those pixels are also marked as being heavily shaded.
    
    Designed for use with EVI mean data to overcome the problem of erroneous high values 
    in shaded mountain valleys in winter.
    
    Values removed are replaced with those from the corresponding "shoulder season" month, 
    whichever is lowest (april or september in north, october or march in the south),
    and the overall standard deviation / mean / count (in band 13) is updated to reflect 
    the changes.
    '''
    
    cdef:
        Py_ssize_t above30N, below30S, y, x, yShape, xShape, zShape
        
        float meanApr, meanSep, meanOct, meanMar
        float sdApr, sdSep, sdOct, sdMar
        int countApr, countSep, countOct, countMar
        float repMean, repSD
        int repCount
        char repSrcMnth
        
        char[:,:,::1] flags
        long long numChanged = 0
        char locationModified = 0
        
        float[:] locationMeans
        float[:] locationSDs
        short[:] locationCounts
    
    flags = np.zeros(shape=(meansData.shape[0],meansData.shape[1],meansData.shape[2]), dtype = np.uint8)
    
    locationMeans = np.zeros(shape=(meansData.shape[0]-1), dtype = np.float32)
    locationSDs = np.zeros(shape=(meansData.shape[0]-1), dtype = np.float32)
    locationCounts = np.zeros(shape=(meansData.shape[0]-1), dtype = np.int16)
    
    yShape = meansData.shape[1]
    xShape = meansData.shape[2]
    zShape = meansData.shape[0]
    print ("{0!s},{1!s},{2!s}".format(zShape,yShape,xShape))
    above30N = 7200
    below30S = 14400
    
    assert meansData.shape[1] == occludedAreas.shape[0]
    assert meansData.shape[2] == occludedAreas.shape[1]
    assert countData.shape[1] == occludedAreas.shape[0]
    assert countData.shape[2] == occludedAreas.shape[1]
    assert sdData.shape[1] == occludedAreas.shape[0]
    assert sdData.shape[2] == occludedAreas.shape[1]
    assert meansData.shape[0] == countData.shape[0]
    assert meansData.shape[0] == sdData.shape[0]
    
    assert NDV < -90
    
    for y in range (yShape):
        if y > above30N and y < below30S:
                # don't do anything within 30deg of equator
                continue
        if y <= above30N:
            for x in range (xShape):
                
                if occludedAreas[y, x] == 0:
                    # don't do anything if we're not on a bit that is predicted to be shadowed
                    continue
                
                locationModified = 0
                
                # Determine the threshold that we will compare winter months to to see if they are 
                # unrealistic
                
                # north of 30N get the min value as lower of april and september
                meanApr = meansData[3, y, x]
                meanSep = meansData[8, y, x]
                sdApr = sdData[3, y, x]
                sdSep = sdData[8, y, x]
                countApr = countData[3, y, x]
                countSep = countData[8, y, x]
                
                # replace ND with a large value so it won't pick up in the less-than test
                if meanApr == NDV:
                    meanApr = 9999
                if meanSep == NDV:
                    meanSep = 9999

                if meanApr < meanSep:
                    # april is lower OR sept is nodata
                    repMean = meanApr
                    repSD = sdApr
                    repCount = countApr
                    repSrcMnth = 4
                
                elif meanSep < meanApr:
                    # sept is lower OR apr is nodata
                    repMean = meanSep
                    repSD = sdSep
                    repCount = countSep
                    repSrcMnth = 9
                    
                elif meanApr == 9999:
                    # both are no data 
                    repMean = 0
                    repSD = 0
                    repCount = 0
                    repSrcMnth = -13
                    
                else:
                    # neither is nodata but they are equal
                    repMean = meanApr
                    # we will assume in this case, although it's not mathematically definite,
                    # that the SDs and counts are also the same
                    repSD = sdApr
                    repCount = countApr
                    repSrcMnth = 13
                
                # Now apply this threshold to oct, nov, dec, jan, feb, mar
                if meansData[0, y, x] > repMean:
                    meansData[0, y, x] = repMean
                    sdData[0, y, x] = repSD
                    countData[0, y, x] = repCount
                    flags [0, y, x] = repSrcMnth
                    locationModified = 1
                    numChanged += 1
                if meansData[1, y, x] > repMean:
                    meansData[1,y, x] = repMean
                    sdData[1, y, x] = repSD
                    countData[1, y, x] = repCount
                    flags [1, y, x] = repSrcMnth
                    locationModified = 1
                    numChanged += 1
                if meansData[2, y, x] > repMean:
                    meansData[2, y, x] = repMean
                    sdData[2, y, x] = repSD
                    countData[2, y, x] = repCount
                    flags [2, y, x] = repSrcMnth
                    locationModified = 1
                    numChanged += 1
                if meansData[9, y, x] > repMean:
                    meansData[9, y, x] = repMean
                    sdData[9, y, x] = repSD
                    countData[9, y, x] = repCount
                    flags [9, y, x] = repSrcMnth
                    locationModified = 1
                    numChanged += 1
                if meansData[10, y, x] > repMean:
                    meansData[10, y, x] = repMean
                    sdData[10, y, x] = repSD
                    countData[10, y, x] = repCount
                    flags [10, y, x] = repSrcMnth
                    locationModified = 1
                    numChanged += 1
                if meansData[11, y, x] > repMean:
                    meansData[11, y, x] = repMean
                    sdData[11, y, x] = repSD
                    countData[11, y, x] = repCount
                    flags [11, y, x] = repSrcMnth
                    locationModified = 1
                    numChanged += 1
                
                if locationModified == 1:
                    locationMeans = meansData[0:zShape-1, y, x]
                    locationSDs = sdData[0:zShape-1, y, x]
                    locationCounts = countData[0:zShape-1, y, x]
                    
                    newAllTimeDailyVals = RegenerateGrandStats(locationMeans, locationSDs, locationCounts, NDV)
                    meansData[zShape-1, y, x] = newAllTimeDailyVals[0]
                    sdData[zShape-1, y, x] = newAllTimeDailyVals[1]
                    countData[zShape-1, y, x] = newAllTimeDailyVals[2]

        elif y >= below30S:
            for x in range (xShape):
                if occludedAreas[y, x] == 0:
                    continue
                
                locationModified = 0
                
                meanOct = meansData[9, y, x]
                meanMar = meansData[2, y, x]
                sdOct = sdData[9, y, x]
                sdMar = sdData[2, y, x]
                countOct = countData[9, y, x]
                countMar = countData[2, y, x]
                
                if meanOct == NDV:
                    meanOct = 9999
                if meanMar == NDV:
                    meanMar = 9999
                
                if meanOct < meanMar:
                    repMean = meanOct
                    repSD = sdOct
                    repCount = countOct
                    repSrcMnth = 10
                    
                elif meanMar < meanOct:
                    repMean = meanMar
                    repSD = sdMar
                    repCount = countMar
                    repSrcMnth = 3
                    
                elif meanOct == 9999:
                    repMean = 0
                    repSD = 0
                    repCount = 0
                    repSrcMnth = -13
                    
                else:
                    repMean = meanOct
                    repSD = sdOct
                    repCount = countOct
                    repSrcMnth = 13
                    
                # apply this threshold to apr, may, jun, jul, aug, sep    
                if meansData[3, y, x] > repMean:
                    meansData[3, y, x] = repMean
                    sdData[3, y ,x] = repSD
                    countData[3, y, x] = repCount
                    flags [3, y, x] = repSrcMnth
                    locationModified = 1
                    numChanged += 1
                if meansData [4, y, x] > repMean:
                    meansData[4, y, x] = repMean
                    sdData[4, y ,x] = repSD
                    countData[4, y, x] = repCount
                    flags [4, y, x] = repSrcMnth
                    locationModified = 1
                    numChanged += 1
                if meansData [5, y, x] > repMean:
                    meansData[5, y, x] = repMean
                    sdData[5, y ,x] = repSD
                    countData[5, y, x] = repCount
                    flags [5, y, x] = repSrcMnth
                    locationModified = 1
                    numChanged += 1
                if meansData [6, y, x] > repMean:
                    meansData[6, y, x] = repMean
                    sdData[6, y ,x] = repSD
                    countData[6, y, x] = repCount
                    flags [6, y, x] = repSrcMnth
                    locationModified = 1
                    numChanged += 1
                if meansData [7, y, x] > repMean:
                    meansData[7, y, x] = repMean
                    sdData[7, y ,x] = repSD
                    countData[7, y, x] = repCount
                    flags [7, y, x] = repSrcMnth
                    locationModified = 1
                    numChanged += 1
                if meansData [8, y, x] > repMean:
                    meansData[8, y, x] = repMean
                    sdData[8, y ,x] = repSD
                    countData[8, y, x] = repCount
                    flags [8, y, x] = repSrcMnth
                    locationModified = 1
                    numChanged += 1
                
                if locationModified == 1:
                    locationMeans = meansData[0:zShape-1, y, x]
                    locationSDs = sdData[0:zShape-1, y, x]
                    locationCounts = countData[0:zShape-1, y, x]
                    
                    newAllTimeDailyVals = RegenerateGrandStats(locationMeans, locationSDs, locationCounts, NDV)
                    meansData[zShape-1, y, x] = newAllTimeDailyVals[0]
                    sdData[zShape-1, y, x] = newAllTimeDailyVals[1]
                    countData[zShape-1, y, x] = newAllTimeDailyVals[2]

    print "Modified {0!s} cell values".format(numChanged)
    #return (np.asarray(meansData), np.asarray(sdData), np.asarray(countData), np.asarray(flags))
    return np.asarray(flags)

Following cell is not used: it was an earlier version of the wintermincut function

In [ ]:
xCorners = np.linspace(0,xSize,10).astype(np.int32)
#yCorners = np.linspace(0,ySize,6).astype(np.int32)
meanVals = np.empty(shape=(ySize, xSize),dtype=np.float32)
countVals = np.empty(shape=(ySize, xSize),dtype='byte')

for x in xrange(len(xCorners)-1):
    x0 = xCorners[x]
    x1 = xCorners[x+1]
    #y0 = yCorners[y]
    #y1 = yCorners[y+1]
    #dataStack = src.read(window=((0,ySize),(x0,x1)), masked=True)
    dataStack= inMeansds.ReadAsArray(x0,0,x1-x0)[:-1]
    #dataStack = np.ma.MaskedArray(dataStackRaw, dataStackRaw == NDV)
    #dataStack[dataStack == NDV] = np.nan
    
    janN = dataStack[0, 0:above30N, :]
    febN = dataStack[1, 0:above30N, :]
    marN = dataStack[2, 0:above30N, :]
    octN = dataStack[9, 0:above30N, :]
    novN = dataStack[10, 0:above30N, :]
    decN = dataStack[11, 0:above30N, :]
    
    # generate a threshold grid as the minimum of apr and sept 
    # Where one of these is nodata, use the other, but when both 
    # are nodata set the output to zero
    # Make copies as we don't want to change the input 
    aprN = np.copy(dataStack[3, 0:above30N, :])
    sepN = np.copy(dataStack[8, 0:above30N, :])
    # minimum ignoring nans unless both are nan
    aprN[aprN == NDV] = np.nan
    sepN[sepN == NDV] = np.nan
    threshN = np.fmin(aprN, sepN)
    #threshN[np.isnan(threshN)] = 0
    threshN = np.nan_to_num(threshN)

    aprS = dataStack[3, below30S:, :]
    mayS = dataStack[4, below30S:, :]
    junS = dataStack[5, below30S:, :]
    julS = dataStack[6, below30S:, :]
    augS = dataStack[7, below30S:, :]
    sepS = dataStack[8, below30S:, :]
    
    octS = dataStack[9, below30S:, :]
    marS = dataStack[2, below30S:, :]
    threshS = np.fmin(octS, marS)
    #threshS[np.isnan(threshS)] = 0
    threshS = np.nan_to_num(threshS)
    
    meanTile = meanVals[:,x0:x1]
    countTile = countVals[:,x0:x1]
    janN[janN > aprN] = 0
    febN[febN > aprN] = 0
    marN[marN > aprN] = 0
    octN[octN > sepN] = 0
    novN[novN > sepN] = 0
    decN[decN > sepN] = 0
    
    mayS[mayS > aprS] = 0
    junS[junS > aprS] = 0
    julS[julS > sepS] = 0
    augS[augS > sepS] = 0
    
    meanTile[:] = np.ma.mean(dataStack[0:12],axis=0)
    countTile[:] = np.logical_not(dataStack.mask).sum(axis=0)
    